In [1]:
import csv

annotation_list = ["Tweet Annotations - Feb Translated Annotations.tsv"]
annotation_list.append("Tweet Annotations - Mar Translated Annotations.tsv")
annotation_list.append("Tweet Annotations - Apr Translated Annotations.tsv")

In [2]:
# Transfer the contents of a tsv file to a list for easier access
def tsv_to_list(annotation_file):
    annotation_list = []
    annotation = open(annotation_file, encoding='utf-8')
    read_tsv = csv.reader(annotation, delimiter="\t") 
    for row in read_tsv:
        annotation_list.append(row)
    return annotation_list

In [3]:
# Get the label for each verse for one annotator
# N: -1, N+NU: -0.5, NU: 0, P+NU: 0.5, P: 1
def get_labels_for(annotation):
    tweets_and_labels = []
    row_num = 0
    for row in annotation:
        if row_num == 0:
            row_num += 1
            continue
        if row[15] != '': # don't use unsure
            row_num += 1
            continue
            #print("not using unsure row", row_num+1)
        if 'UNUSABLE' in row[16].upper() or 'NOT ENOUGH INFO' in row[16].upper():
            row_num += 1
            continue
            #print("unusable row", row_num+1)
        elif row[6].upper() == 'X' and (row[8].upper() != 'X' and row[7].upper() == 'X'): # don't use pos + neg mixed labels
            row_num += 1
            continue
            #print("not using pos+neg row", row_num+1)
        elif row[6].upper() == 'X' and (row[7].upper() != 'X' and row[8].upper() != 'X'): # only pos
            new_row = [annotation[row_num][0], annotation[row_num][1], annotation[row_num][2], annotation[row_num][3], annotation[row_num][4], annotation[row_num][5], 1]
            tweets_and_labels.append(new_row)
        #elif row[6].upper() == 'X' and (row[8].upper() == 'X' and row[7].upper() != 'X'): # pos + neu
        #    new_row = [annotation[row_num][0], annotation[row_num][1], annotation[row_num][2], annotation[row_num][3], annotation[row_num][4], annotation[row_num][5], 3]
        #    tweets_and_labels.append(new_row)
        elif row[8].upper() == 'X' and (row[6].upper() != 'X' and row[7].upper() != 'X'): # only neu 
            new_row = [annotation[row_num][0], annotation[row_num][1], annotation[row_num][2], annotation[row_num][3], annotation[row_num][4], annotation[row_num][5], 0]
            tweets_and_labels.append(new_row)
        elif row[7].upper() == 'X' and (row[8].upper() != 'X' and row[6].upper() != 'X'): # only neg
            new_row = [annotation[row_num][0], annotation[row_num][1], annotation[row_num][2], annotation[row_num][3], annotation[row_num][4], annotation[row_num][5], -1]
            tweets_and_labels.append(new_row)
        #elif row[8].upper() == 'X' and (row[7].upper() == 'X' and row[6].upper() != 'X'): # neg + neu
        #    new_row = [annotation[row_num][0], annotation[row_num][1], annotation[row_num][2], annotation[row_num][3], annotation[row_num][4], annotation[row_num][5], 2]
        #    tweets_and_labels.append(new_row) 
        #else:
        #    print("outlier row:", row_num+1)
        row_num += 1
    return tweets_and_labels

In [4]:
feb_list = tsv_to_list(annotation_list[0])
feb_labels = get_labels_for(feb_list)
mar_list = tsv_to_list(annotation_list[1])
mar_labels = get_labels_for(mar_list)
apr_list = tsv_to_list(annotation_list[2])
apr_labels = get_labels_for(apr_list)

all_labels = feb_labels + mar_labels + apr_labels
#print(all_labels.count(-1), all_labels.count(0), all_labels.count(1))

import pandas as pd
data = pd.DataFrame(all_labels, columns=['tweetID', 'tweetTime', 'tweetText', 'tweetLang', 'tweetCoordinates', 'tweetPlace', 'tweetSentiment'])
data.head()

test_data = pd.DataFrame(apr_labels, columns=['tweetID', 'tweetTime', 'tweetText', 'tweetLang', 'tweetCoordinates', 'tweetPlace', 'tweetSentiment'])

# Drop 'airline' column
#data.drop(['airline'], inplace=True, axis=1)

In [5]:
#!pip3 install nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import PunktSentenceTokenizer


In [6]:
# Get all the words from the data
import random

text = []
for tweet in data['tweetText'].values:
    text.append(tweet.split())

documents = [(words, label) for words in text for label in data['tweetSentiment']]
print(documents[0])

random.shuffle(documents)

(['I', 'love', 'to', 'look', 'at', "people's", 'opinions', 'about', 'the', 'coronavirus', 'because', 'I', 'can', 'find', 'both', 'memes', 'and', 'people', 'who', 'believe', 'that', 'this', 'will', 'be', 'World', 'War', 'Z', 'I', 'also', 'love', 'Spain', 'for', 'doing', 'these', 'trends', 'tt'], 1)


In [8]:
all_words = []

for sent in text:
    for w in sent:
        all_words.append(w.lower())

all_words = FreqDist(all_words)

word_features = list(all_words.keys())[:3000]
print(word_features[:20])

['i', 'love', 'to', 'look', 'at', "people's", 'opinions', 'about', 'the', 'coronavirus', 'because', 'can', 'find', 'both', 'memes', 'and', 'people', 'who', 'believe', 'that']


In [9]:
def find_features(document):
    words = set(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)

    return features

In [10]:
featuresets = [(find_features(rev), category) for (rev, category) in documents]

MemoryError: 

In [ ]:
example_sent = "This is a sample sentence, showing off the stop words filtration."

stop_words = set(stopwords.words('english'))

word_tokens = word_tokenize(example_sent)

filtered_sentence = [w for w in word_tokens if not w in stop_words]

filtered_sentence = []

for w in word_tokens:
    if w not in stop_words:
        filtered_sentence.append(w)


In [ ]:
ps = PorterStemmer()
for w in example_words:
    print(ps.stem(w))